## 3.3 Experiments – 20 marks

### 3.3.1 Design experiments to test the following: The utility of the data that you have generated using your proposed anonymisation scheme (algorithms) for Q2.c.


In [4]:
import pandas as pd

# Assuming df is your pandas DataFrame
cardinalities = {}
df = pd.read_csv('your_anonymized_dataset.csv')
for column in df.columns:
    u = df[column].nunique()  # Number of unique entries in the column
    n = df[column].count()  # Total number of non-NA/null values in the column
    c = u / n  # Cardinality calculation
    cardinalities[column] = c

# Print the cardinality for each attribute
for attribute, cardinality in cardinalities.items():
    print(f"The cardinality of {attribute} is: {cardinality}")

The cardinality of manner_of_death is: 0.00025034422330704717
The cardinality of armed is: 0.013628182051941374
The cardinality of gender is: 0.00025131942699170643
The cardinality of race is: 0.0009269272362119574
The cardinality of state is: 0.006383777694329703
The cardinality of signs_of_mental_illness is: 0.00025034422330704717
The cardinality of threat_level is: 0.0003755163349605708
The cardinality of flee is: 0.0005688282138794084
The cardinality of body_camera is: 0.00025034422330704717
The cardinality of age_range is: 0.0004005875283749499
The cardinality of year_range is: 0.0003755163349605708


### 3.3.2 Design experiments to test the following: Analyse the new (anonymized) dataset for risks of de-anonymization.
We are using the same Algorithm proposed in Task 3.2

In [41]:
import pandas as pd
from scipy import stats
import numpy as np
import prince
import warnings
import altair as alt
alt.data_transformers.enable("vegafusion")

df = pd.read_csv('your_anonymized_dataset.csv')
# Ignore specific PerformanceWarnings from pandas
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
# Load the dataset

mca = prince.MCA(
    n_components=3,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
# Fit FAMD on the dataset
mca = mca.fit(df)

df_transformed = mca.transform(df)
z_scores = np.abs(stats.zscore(df_transformed))
outliers = np.where(z_scores > 3) 

outlier_rows = df.iloc[outliers[0]]
outlier_rows.to_csv('athletes_outliers_MCA.csv', index=False)

mca.plot(
    df,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=True,
    show_column_labels=False,
    show_row_labels=False
)

alt.LayerChart(...)

### 3.3.3 Design experiments to test the following: Propose a method of assessing the risk of disclosure (de-anonymisation) and use this metric to evaluate your anonymised datasets (from Assignments #1, and #2-3), the anonymised dataset received from your colleague, and your version of the anonymised dataset that you obtained in Q2.c.
